3 types of MPI names

- Datatype: such as MPI_Info, MPI_Win. It is MPI_ followed by a noun with first capitalized letter
- Function: such as MPI_Init, MPI_Send. It is MPI_ followed by a verb with first capitalized letter
- Constant: such as MPI_COMM_WORLD, MPI_INT. It is MPI_ followed by a noun with all letters capitalized



# MPI SYSTEM and Communication

More context here:

https://www.codingame.com/playgrounds/349/introduction-to-mpi/mpi_comm_world-size-and-ranks

- MPI_Init(&argc, &argv): Communicator set up 

- MPI_Comm_size(MPI_COMM_WORLD, &size): Get the number of processes

- MPI_Comm_rank(MPI_COMM_WORLD, &rank): Get the rank of the process

- MPI_Finalize(): Finalize the MPI environment

So far we have used the default communicator only
MPI_Comm comm = MPI_COMM_WORLD;
But you can do much more with them, and here we just give a short introduction to those possibilities

Duplicate

Split

Define new communicators by groups of processes

Spawn new communicators (highly advanced MPI)

Intercommunicate (highly advanced MPI)

Duplicating

- int **MPI_Comm_dup**(MPI_Comm comm, MPI_Comm *newcomm)

- int **MPI_Comm_idup**(MPI_Comm comm, MPI_Comm *newcomm, MPI_Request *request)

- int **MPI_Comm_free**(MPI_Comm *comm);

Splitting 

- int **MPI_Comm_split**( MPI_Comm comm, int color, int key, MPI_Comm *newcomm)

comm: communicator (handle)

color: control of subset assignment (integer)

key: control of rank assignment (integer)

newcomm: new communicator (handle)

Constructing new by groups and get group of communicators

- int **MPI_Comm_group**(MPI_Comm comm, MPI_Group *group)

comm : Communicator (handle)

group : Group in communicator (handle)

• Manipulate the groups with functions like MPI_Group_incl, MPI_Group_excl, …

• Create the communicator(s) by

- int **MPI_Comm_create**( MPI_Comm comm, MPI_Group group, MPI_Comm *newcomm )

newcomm : new communicator (handle).

- int **MPI_Group_incl**(MPI_Group group, int n, const int ranks[], MPI_Group *newgroup)

group Group (handle).

n Number of elements in array ranks (and size of newgroup)(integer).
ranks Ranks of processes in group to appear in newgroup (array of
integers)

- int **MPI_Group_excl**(MPI_Group group, int n, const int ranks[], MPI_Group *newgroup)

group Group (handle).

n Number of elements in array ranks (integer). 

ranks Array of integer ranks in group not to appear in newgroup.

Using groups to improve one-sided comms

• Define exposure epoch, on target, and access epoch, on origin,
epochs using process groups

• Target runs exposure epoch by issuing

- int **MPI_Win_post**(MPI_Group group, int assert, MPI_Win win)

- int **MPI_Win_wait**(MPI_Win win)

• Origin runs access epoch by issuing

- int **MPI_Win_start**(MPI_Group group, int assert, MPI_Win win)

- int **MPI_Win_complete**(MPI_Win win)

![image.png](SendReceive.png)

# MPI SEND CLASS

- int **MPI_Send**(const void* buf, int count, MPI_Datatype datatype, int dest,int tag, MPI_Comm comm) 

- int **MPI_Ssend**(const void *buf, int count, MPI_Datatype datatype, int dest, int tag, MPI_Comm comm)

• ”S” for “Synchronous”, meaning that the receiver is always forced to send an acknowledge. It will not avoid deadlocks. In this case, all unsafe operations should always deadlock, helping you out to debug and write “safer” code

- int **MPI_Bsend**(const void *buf, int count, MPI_Datatype datatype, int dest, int tag, MPI_Comm comm)

User is responsible for allocating large enough buffers

- int **MPI_Isend**(const void *buf, int count, MPI_Datatype datatype, int dest,
int tag, MPI_Comm comm, MPI_Request *request)

Immediate or Incomplete. “Here is my data, please send it forward as I instruct” or “I am expecting certain type of data to come to this provided buffer space”.

- int **MPI_Rsend**(const void *buf, int count, MPI_Datatype datatype, int dest, int tag, MPI_Comm comm)

R for “Ready”. It is a blocking send, but it will not start until the matching receive is posted. It will not avoid deadlocks.

- int **MPI_Sendrecv**(const void *sendbuf, int sendcount, MPI_Datatype sendtype,
    int dest, int sendtag, void *recvbuf, int recvcount,
    MPI_Datatype recvtype, int source, int recvtag,
    MPI_Comm comm, MPI_Status *status): 

Sends and receives a message with the right choice of source and destination. Then you always need a “pair” to communicate with. If not, then you need to use “MPI_PROC_NULL”

# MPI RECEIVE CLASS

- int **MPI_Recv**(void* buf, int count, MPI_Datatype datatype, int source,int tag, MPI_Comm comm, MPI_Status *status)

- int **MPI_Irecv**(void *buf, int count, MPI_Datatype datatype, int source, int tag, MPI_Comm comm, MPI_Request *request)

Immediate or Incomplete Non-blocking routines yield an MPI_Request object. This request can then
be used to query whether the operation has completed. MPI_Irecv routine
does not yield an MPI_Status object. This is because the status object
describes the actually received data, and at the completion of
the MPI_Irecv call there is no received data yet.

# One sided communication typical workflow

- **MPI_Info** info;

Declares an MPI_Info object.

Description: MPI_Info is used to pass additional information or hints to the MPI runtime about how the communication should be optimized. It's like a set of key-value pairs. In many cases, it can just be set to MPI_INFO_NULL if no special options are needed.

- **MPI_Win** window;

Declares an MPI_Win object.

Description: This object represents the memory window used in one-sided communication. A window is a region of memory that is made available for direct access by other processes.

- int **MPI_Win_create**(void *base, MPI_Aint size, int disp_unit, MPI_Info info, MPI_Comm comm, MPI_Win *win);

Allocates window segment

Purpose: Creates an MPI basic window object for one-sided communication.

base (pointer to) local memory to expose for RMA (remote memory access) 

size of a local window in bytes

disp_unit local unit size for displacements in bytes

info info argument

comm communicator

win handle to window

- int **MPI_Win_create_dynamic**(MPI_Info info, MPI_Comm comm, MPI_Win *win)

Similar to the others, but only a pointer to a window object, with
its attached memory yet unspecified, is returned.
to an empty buffer is returned. To attach memory to the Window:

- **MPI_Win_attach**(MPI_Win win, void *base, MPI_Aint size)

- **MPI_Win_detach**(MPI_Win win, void *base)


- int **MPI_Win_allocate**(MPI_Aint size, int disp_unit, MPI_Info info, MPI_Comm comm, void *baseptr, MPI_Win *win)

size: size of a local window in bytes

disp_unit local unit size for displacements, in bytes

info: info argument

comm: communicator

baseptr: address of local allocated window segment

win: window object returned by the call

- int **MPI_Win_free**(MPI_Win *win) ;

Purpose: Frees the MPI window object.

Description: This function is called when the window object is no longer needed. It deallocates any resources associated with the window and makes the memory region inaccessible for one-sided communication.


Active (global) RMA synchronization: fences
- **MPI_Win_fence** (assert, MPI_Win win)

assert optimize for specific usage. Valid values are ”0”,
MPI_MODE_NOSTORE, MPI_MODE_NOPUT,
MPI_MODE_NOPRECEDE, MPI_MODE_NOSUCCEED

win window handle

• Used both starting and ending an epoch
• Assertions with 0 will always work, but being more specific could help, see the next slide (advanced)

Passive synchronization
- int **MPI_Win_lock**(int lock_type, int rank, int assert, MPI_Win win)

lock_type: Indicates whether other processes may access the target window at the same time (if MPI_LOCK_SHARED) or not (MPI_LOCK_EXCLUSIVE)

rank: rank of the process having the locked (target) window

assert: Used to optimize this call; zero may be used as a default.

win: window object

- int **MPI_Win_unlock**(int rank, MPI_Win win)


- int **MPI_Put**(const void *origin_addr, int origin_count, MPI_Datatype
            origin_datatype, int target_rank, MPI_Aint target_disp,
            int target_count, MPI_Datatype target_datatype, MPI_Win win)

Moving data: put

The sending process writes data directly to the memory of the receiving process. The receiver does not need to post a corresponding receive.

• Otherwise very normal-looking call, but the
target data description is somewhat non-trivial

• When creating a window, you need to specify the
displacement unit (from the window start)

- int **MPI_Get**(void *origin_addr, int origin_count, 
    MPI_Datatype origin_datatype, int target_rank, 
    MPI_Aint target_disp, int target_count, 
    MPI_Datatype target_datatype, MPI_Win win)

Similar syntax to MPI_Put

The sending process reads data directly from the memory of the receiving process. Again, the receiving process is not actively involved in the operation.

- int **MPI_Accumulate**(const void *origin_addr, int origin_count,
    MPI_Datatype origin_datatype, int target_rank,
    MPI_Aint target_disp, int target_count,
    MPI_Datatype target_datatype, MPI_Op op, MPI_Win win)

• Store data from the origin process to the memory window of the target process and combine it using one of the predefined MPI reduction operations

• Predefined operators are available (we talk about these in the connection of collectives), but no user-defined ones.

• There is one extra operator: MPI_REPLACE, this has the effect that only the last result to arrive is retained.


int **MPI_Get_accumulate**(const void *origin_addr, int
origin_count,MPI_Datatype origin_datatype, void
*result_addr, int result_count, MPI_Datatype result_datatype,
int target_rank, MPI_Aint target_disp, int target_count,
MPI_Datatype target_datatype, MPI_Op op, MPI_Win win)

• Store data from target window to the origin, and combine it with the predefined
operation.

• Predefined operators are available (we talk about these in the connection of
collectives), but no user-defined ones.

• There is one extra operator: MPI_REPLACE, this has the effect that only the
last result to arrive is retained.


# Collective communications

- int **MPI_Bcast**(void* buffer, int count, MPI_Datatype datatype, int root, MPI_Comm comm)

Collective data movement; Broadcast

These two processes are reverse of each other

- int **MPI_Gather**( const void* sendbuf, int sendcount, MPI_Datatype sendtype,
void* recvbuf, int recvcount, MPI_Datatype recvtype, int root, MPI_Comm comm)

- int **MPI_Scatter** (void* sendbuf, int sendcount, MPI_Datatype sendtype,
void* recvbuf, int recvcount, MPI_Datatype recvtype, int root, MPI_Comm comm)

• Send and receive buffers are no longer of the same size, hence need to specify two buffers

• Root receives/sends 𝑛𝑝 sized buffer of data, others send/receive data of the size 𝑛.

• Counterintuitively, root’s recvcount/sendcount is NOT 𝑛𝑝, but 𝑛.

• SPMD code; everybody will have to allocate the large buffer; is that not awkward? Yes, other than ‘root’ processes,

• use a null pointer in place of the larger buffer

• Or use the option “MPI_IN_PLACE” for the unnecessary buffers.

- int **MPI_Allgather**(const void *sendbuf, int sendcount, MPI_Datatype sendtype,
void *recvbuf, int recvcount, MPI_Datatype recvtype, MPI_Comm comm)

- int **MPI_Iallgather**(const void *sendbuf, int sendcount, MPI_Datatype sendtype,
void *recvbuf, int recvcount, MPI_Datatype recvtype, MPI_Comm comm, MPI_Request *request)

- int **MPI_Alltoall**(const void *sendbuf, int sendcount, MPI_Datatype sendtype,
void *recvbuf, int recvcount, MPI_Datatype recvtype, MPI_Comm comm)

- int **MPI_Ialltoall**(const void *sendbuf, int sendcount, MPI_Datatype sendtype,
void *recvbuf, int recvcount, MPI_Datatype recvtype, MPI_Comm comm, MPI_Request *request)

The basic routines send/receive the same amount of data from each process
• “v” for vector routines to allow the programmer to specify a message of different length for each destination (one-to-all) or source (all-toone) or destination and source (all-to-all)
• May need to use some other collectives to compute the required displacements

- int **MPI_Gatherv**(const void *sendbuf, int sendcount, MPI_Datatype sendtype, void *recvbuf,
const int recvcounts[], const int displs[], MPI_Datatype recvtype, int root, MPI_Comm comm)

- int **MPI_Alltoallv**(void *sendbuf, int *sendcnts, int *sdispls, MPI_Datatype sendtype,
void *recvbuf, int *recvcnts, int *rdispls, MPI_Datatype recvtype, MPI_Comm comm)

- int **MPI_Reduce**(const void *sendbuf, void *recvbuf, int count,
MPI_Datatype datatype, MPI_Op op, int root, MPI_Comm comm)

- int **MPI_Ireduce**(const void *sendbuf, void *recvbuf, int count,
MPI_Datatype datatype, MPI_Op op, int root, MPI_Comm comm, MPI_Request *request)

- int **MPI_Allreduce**(const void* sendbuf, void* recvbuf, int count,
MPI_Datatype datatype, MPI_Op op, MPI_Comm comm)

- int **MPI_Iallreduce**(const void *sendbuf, void *recvbuf, int count,
MPI_Datatype datatype, MPI_Op op, MPI_Comm comm, MPI_Request *request)

- int **MPI_Scan**(const void *sendbuf, void *recvbuf, int count, MPI_Datatype datatype, MPI_Op op,
MPI_Comm comm)

- int **MPI_Iscan**(const void *sendbuf, void *recvbuf, int count, MPI_Datatype datatype, MPI_Op op,
MPI_Comm comm, MPI_Request *request)

- int **MPI_Exsca**(const void *sendbuf, void *recvbuf, int count, MPI_Datatype datatype, MPI_Op op,
MPI_Comm comm)

- int **MPI_Iexscan**(const void *sendbuf, void *recvbuf, int count, MPI_Datatype datatype, MPI_Op op,
MPI_Comm comm, MPI_Request *request)

- int **MPI_Reduce_scatter**(const void *sendbuf, void *recvbuf, const int recvcounts[],MPI_Datatype
datatype, MPI_Op op, MPI_Comm comm)

- int **MPI_Ireduce_scatter**(const void *sendbuf, void *recvbuf, const int recvcounts[],MPI_Datatype
datatype, MPI_Op op, MPI_Comm comm, MPI_Request *request)

MPI_Op takes these values:

MPI type meaning applies to\

MPI_MAX maximum integer, floating point

MPI_MIN minimum

MPI_SUM sum integer, floating point, complex, multilanguage types

MPI_REPLACE overwrite

MPI_NO_OP no change

MPI_PROD product

MPI_LAND logical and C integer, logical

MPI_LOR logical or

MPI_LXOR logical xor

MPI_BAND bitwise and integer, byte, multilanguage types

MPI_BOR bitwise or

MPI_BXOR bitwise xor

MPI_MAXLOC max value and location MPI_DOUBLE_INT and such

MPI_MINLOC min value and location

User defined operation

- int **MPI_Op_create**(MPI_User_function *function, int commute, MPI_Op *op)

Prototype 

- typedef void **MPI_User_function**(void *invec, void *inoutvec, int *len,
MPI_Datatype *datatype);

inoutvec[i] = invec[i] op inoutvec[i] from i=0;len-1

• The operation is assumed to be associative

• You can use flag “commute” to indicate whether the function is in
addition commutative or not.

• Void return as no errors are expected


# Syncronization

int **MPI_Barrier**(MPI_Comm comm)

int **MPI_Ibarrier**(MPI_Comm comm, MPI_Request *request)

• Waits until all processes have called it

• Forces time synchronization

• Not needed very often, as collectives impose synchronization on their own

# User defined data types

Defining and decommissioning a new datatype

- int **MPI_Type_contiguous**(int count,MPI_Datatype oldtype, MPI_Datatype *newtype)

- int **MPI_Type_XXX**(…MPI_Datatype oldtype, MPI_Datatype *newtype)

- int **MPI_Type_commit**(MPI_Datatype *newtype)…

- int **MPI_Type_free**(MPI_Datatype *newtype)

- int **MPI_Type_vector**(int count, int blocklength, int stride, MPI_Datatype oldtype, MPI_Datatype *newtype)

count number of blocks

blocklength number of replicated oldtype elements in each block

stride total number of elements in each block

newtype the new datatype to commit, use, and decommission

oldtype the datatype to use for constructing newtype

count number of replicas

XXX stands for one of the constructors

- int **MPI_Type_create_subarray**(int ndims, const int sizes[], const int subsizes[], 
const int offsets[], int order, MPI_Datatype oldtype, MPI_Datatype *newtype)

ndims number of array dimensions

sizes number of array elements in each dimension

subsizes number of subarray elements in each dimension

offsets starting point of subarray in each dimension

order storage order of the array. Either MPI_ORDER_C or MPI_ORDER_FORTRAN

---

Datatype constructors Datatype constructors

MPI_Type_contiguous contiguous datatypes

MPI_Type_vector regularly spaced datatype

MPI_Type_indexed variably spaced datatype

MPI_Type_create_subarray subarray within a multi-dimensional array

MPI_Type_create_hvector like vector, but uses bytes for spacings

MPI_Type_create_hindexed like index, but uses bytes for spacings

MPI_Type_create_struct fully general datatype

newtype the new datatype to commit, use, and
decommission
oldtype the datatype to use for constructing newtype



# Topology

Cartesian grid topology

- int **MPI_Cart_create**( MPI_Comm comm_old, int ndims, const int
dims, const int periods, int reorder, MPI_Comm *comm_cart);

- int **MPI_Cart_coords**( MPI_Comm comm, int rank, int maxdims, int coords);

- int **MPI_Cart_rank**( MPI_Comm comm, init coords, int *rank);

ndims f.ex. 2 for 2-dim, 3 for 3-dim

dims of grid in each ndim (size of ndim)

periods which of the boundaries are periodic?

reorder can MPI re-order ranks as to what it sees optimal?

coords Coordinate of the process in the Cartesian topology

rank the rank of the process in the Cartesian topology

- int **MPI_Cart_shift**(MPI_Comm comm, int direction, int displ, int *source, int *dest)

Determine the neighbors for communication

direction Shifting direction in the defined dim

displ displacement in ranks >0 for up <0 down in the direction

source Neighbor rank in decreasing index

dest Neighbor rank towards increasing index

“Names” of the neighbor ranks come from MPI_Sendrecv, in the context of which this routine is often used

- int **MPI_Dist_graph_create_adjacent**(MPI_Comm oldcomm, int indegree, int sources[], int sourceweights[],
int outdegree, int dests[], int destweights[], MPI_Info info, int reorder, MPI_Comm *newcomm)

indegree : number of source nodes; 

sources : array containing the ranks of the source nodes; 

sourceweights : weights for source to destination edges or MPI_UNWEIGHTED; 

outdegree : array specifying the number of destinations, 

dests : ranks of the destination nodes, 

destweights : weights for destination to source edges or MPI_UNWEIGHTED; 

info : hints on optimization and interpretation of weights

reorder : the process may be reordered?

- int **MPI_Dist_graph_create**(MPI_Comm comm_old, int n, const int sources[], const int degrees[], const int destinations[], const int weights[], MPI_Info info, int reorder, MPI_Comm *comm_dist_graph)

n: number of source nodes; sources : array containing the ranks of the
source nodes; 

degrees : array specifying the number of destinations for
each source node, 

destinations : ranks of the destination nodes,

weights : weights for destination to source edges or
MPI_UNWEIGHTED; 

info : hints on optimization and interpretation of
weights, 

reorder : the process may be reordered?

- int **MPI_Dist_graph_neighbors**(MPI_Comm comm, int maxindegree, int sources[],
int sourceweights[], int maxoutdegree, int destinations[], int destweights[])

- int **MPI_Dist_graph_neighbors_count**(MPI_Comm comm, int *indegree, int *outdegree, int *weighted)

